In [ ]:
import os 
import glob
import pandas as pd 
import numpy as np
from utils.linkageTree import linkageCut
from tsp.TSP_Formulation_Methods import ( 
    create_QUBO_matrix,
    solve_qubo_with_Dwave,
    check_solution, 
    load_lambda_means,
    draw_solution_graph,
)
from vqaa.vqaa_tools import ( 
    heuristical_embedding, 
    atoms_register,
    atoms_list, 
    generate_grid,
    run_vqaa,
    plot_distribution,
)
from utils.utils import ( 
    view_linkage_on_map, 
    draw_centers_on_map,
    map_draw_line,
    convert_bitstring_to_matrix
)

# Load initial data 

In [ ]:
# Load previously stored overpy lat/lon datafile for different amenities
amenities_data = pd.read_csv(os.path.join(os.pardir, 'data', 'amenities-granada.csv'))
# If there is no previous data:
# amenities_data = utils.fetch_amenities_from() # Defaults to Granada
# Create a hierarchical clustering of amenities
hierarchical_cluster = linkageCut(amenities_data)
# Set a specific number of clusters per levels
nclusters = 6
levels = 2
labels = hierarchical_cluster.top_down_view_recur(nclusters=6, levels=2)
# Visualize for debugging purposes.
view_linkage_on_map(linkage_matrix=hierarchical_cluster)

# First iteration

In [ ]:
# Fetch the centers of the first level
centers = hierarchical_cluster.give_centers_level(0)
# Sanity check by drawing the graph
draw_centers_on_map(centers)

In [ ]:
# Fetch the distance from the centers of the first level
distances = hierarchical_cluster.dist_matrix_level(0, return_labels=False)
# Set initial global parameters
N = distances.shape[0]
p = 3

startNode = 0
endNode = np.random.choice(np.arange(distances.shape[0] - 1) + 1)

# Process Parameters
p = min(p, N-1)
startNode = min(startNode, N-1)
endNode = min(endNode, N-1)

reduced_distances = distances[:N,:N]/np.max(distances[:N,:N])
maxDistance = np.max(reduced_distances)
# NOTE: temporary double pardir while we decide the new structure for the lambdas
lambda_paths = glob.glob(os.path.join(os.path.pardir, os.path.pardir, 'data', 'lamdasOptimized', '*'))
mean_lambdas = load_lambda_means(lambda_paths)

# Formulation with initial lambdas
Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode, endNode, mean_lambdas)

In [ ]:
solution_Dwave, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
print("\nD-Wave solution:")
print(solution_Dwave)
check_solution(solution_Dwave, N,  p, startNode, endNode)

In [ ]:
adjacency = convert_bitstring_to_matrix(solution_Dwave, N=N, p=p)
map_draw_line(centers[:, ::-1], adjacency, color='blue')

In [ ]:
# coords = heuristical_embedding(atoms_list(len(Q_matrix_initial)), generate_grid(50, 50,1), Q_matrix_initial)
# register = atoms_register(coords)

In [ ]:
# Why is it takiung > 5minutes????
# C, x = run_vqaa(Q_matrix_initial, register, "mps")

# Second iteration

In [ ]:
connections = np.concatenate([adjacency[:,0].nonzero()[0], adjacency[0, :].nonzero()[0]], axis=0)
# Fetch the centers of the first level
distances, indices, _ = hierarchical_cluster.dist_matrix_label_down(
    1,
    connections=connections,
)
print(distances)

In [ ]:
# Formulation with initial lambdas
choices = set(np.arange(distances.shape[0] - 1) + 1)
print(choices)
startNode = None
if len(indices) == 1:
    startNode = indices[0]
if len(indices) == 2:
    endNode = indices[1]
if startNode:
    choices = choices - set([startNode])
if not endNode:
    endNode = np.random.choice(choices)

print(startNode)
print(endNode)

Q_matrix_initial,_ = create_QUBO_matrix(reduced_distances, p, startNode, endNode, mean_lambdas)
solution_Dwave, _ = solve_qubo_with_Dwave(Q_matrix_initial, num_reads=1000)
print("\nD-Wave solution:")
print(solution_Dwave)
check_solution(solution_Dwave, N,  p, startNode, endNode)
next_adjacency = convert_bitstring_to_matrix(solution_Dwave, N=N, p=p)

In [ ]:
next_centers = hierarchical_cluster.give_centers_label_down(1)
map = map_draw_line(centers[:, ::-1], adjacency, color='blue')
map_draw_line(next_centers[:, ::-1], next_adjacency, color='red', map=map)